In [23]:
cd ../

C:\Users\rrrod


In [35]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Path to the folder containing the data files
folder_path = "FFC_4/data/obs_042424/Phase1"

# Initialize lists to store frequency and intensity data
frequency_data = []
intensity_data = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".npz"):  # Assuming data files have a .npz extension
        file_path = os.path.join(folder_path, filename)
        try:
            # Load data from the npz file
            data = np.load(file_path)
            # Extract frequency and intensity arrays from the npz file
            frequency = data['frequency']
            intensity = data['intensity']
            # Append data to lists
            frequency_data.append(frequency)
            intensity_data.append(intensity)
        except Exception as e:
            print(f"Error loading data from {file_path}: {e}")

# Check if any data was loaded
if not frequency_data or not intensity_data:
    print("No valid data loaded. Check the data files.")
else:
    # Concatenate frequency and intensity arrays
    frequency_data = np.concatenate(frequency_data)
    intensity_data = np.concatenate(intensity_data)

    # Plot the H_1 21 cm line
    plt.figure(figsize=(10, 6))
    plt.plot(frequency_data, intensity_data, color='blue')
    plt.title("H_1 21 cm Line")
    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Intensity")
    plt.grid(True)
    plt.show()


Error loading data from FFC_4/data/obs_042424/Phase1\spec100_L180_B54.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec101_L161_B51.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec102_L175_B54.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec103_L134_B37.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec104_L138_B40.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec105_L170_B54.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec106_L143_B44.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec107_L157_B51.npz: 'frequency is not a file in the archive'
Error loading data from FFC_4/data/obs_042424/Phase1\spec109_L166_B54.np